Improved segmentation with morphological transforms and watershedding

In [2]:
import numpy as np
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io as skio
from skimage import filters, exposure, measure, draw, transform, segmentation, morphology, feature, color
import parse
import os
import re
from scipy import ndimage
import h5py
import colorcet as cc
import cv2

In [3]:
import sys

In [4]:
sys.version_info[:]

(3, 9, 12, 'final', 0)

In [14]:
rootdir = "/home/no221/test"
print(rootdir)

/home/no221/test


In [6]:
# Set pixel sizes (from raw images) for downsampling (changed from 0.6061 from Harvard and downsample=2)
downsample = 1
dx = 1.2133*downsample
dz = 0.9999*downsample

In [7]:
def parse_directories(rootdir, sample_regex, capture_regexes):
    """ Find files containing data and extract metadata from file names, converting this into a pandas dataframe that can be referenced.
    """
    # Find the last subdirectory that we consider part of the root
    last_rootdir = os.path.split(rootdir)[1]
    data_dict = {}
    
    # Recursively walk the root directory
    for root, dirnames, fnames in os.walk(rootdir):
        if len(fnames) == 0:
            continue
        # find individual samples according to a rule
        for f in fnames:
            m = re.search(sample_regex, f)
            if m:
                sample_name = m.group(0)
                sample_path = os.path.join(root, sample_name)
                if sample_path not in data_dict:
                    data_dict[sample_path] = {}
                    data_dict[sample_path]["sample_name"] = sample_name
                    data_dict[sample_path]["file_path"] = os.path.join(root, f)
                    dirsplit = root.split("/")
                    root_index = dirsplit.index(last_rootdir)
                    subfolders = dirsplit[root_index+1:]
#                     print(subfolders)
                    # Extract information from subfolder structure (e.g. probability, dpf, etc.)
                    for sfi, sf in enumerate(subfolders):
                        data_dict[sample_path]["subfolder_%d" % sfi] = sf
                    
                    # Extract user-defined columns from fielnames
                    for f2 in fnames:
                        if sample_name in f2:
                            for cr in capture_regexes:
                                n = re.search(cr % sample_name, f2)
                                if n:
                                    data_dict[sample_path][n.group(0)] = os.path.join(root, f2)
    df = pd.DataFrame(data_dict).transpose().rename_axis("sample_id")
    return df

In [8]:
def mask_to_labels(mask_img, bg_class):
    """ Convert a single image mask with different Ilastik classifications to
        a list of label images for each class (excluding the background) with
        different numbers for each distinct ROI.
    """
    classes = np.unique(mask_img)
    label_images = []
    for cl in classes:
        if cl == bg_class:
            continue
        label_images.append(ndimage.label(mask_img==cl)[0])
    return label_images

In [9]:
def ellipse_3d(region_mask):
    """ Calculate coordinates of an ellipse with principal axes equal to the region
        marked by `region_mask`.
    """
    coords = np.argwhere(region_mask)
    centroid = np.mean(coords, axis=0)
    norm_coords = (coords-centroid).astype(np.float32)
    norm_coords*= np.array([dz,dx,dx]).astype(np.float32)
    covariance_matrix = (norm_coords.T @ norm_coords)/norm_coords.shape[0]
    w, v = np.linalg.eig(covariance_matrix)
    ellipse_volume = 4/3*np.pi*np.prod(w)
    ellipse_area = 4*np.pi*(((w[0]*w[1])**1.6 + (w[0]*w[2])**1.6 + (w[2]*w[1])**1.6)/3)**(1/1.6)
    eigenvalues = tuple(list(w))
    return eigenvalues + (ellipse_area,) + (ellipse_volume,)

In [10]:
def simple_probabilities_to_mask_array(probability_img, bg_class):
    """ Convert a ZxYxXxC image of Ilastik probabilities to a list of binary masks
        for each class based on the maximum likelihood at each pixel.
    """
    mask_images = []
    for cl in range(probability_img.shape[-1]):
        if cl == (bg_class-1):
            continue
        ML_class = np.argmax(probability_img, axis=-1) == cl
        mask_images.append(ML_class)
    return np.array(mask_images).astype(bool)

def simple_mask_array_to_labels(mask_images, bg_class):
    """ Take a CxZxYxX list of binary masks for each Ilastik classification and label
        distinct ROIs.
    """
    label_images = []
    for cl in range(mask_images.shape[0]):
        if cl == (bg_class-1):
            continue
        label_images.append(ndimage.label(mask_images[cl])[0])
    return np.array(label_images)

def contact_area_5(region_mask, distance):
    """ Identify pixels within `region_mask` with a distance (to blood vessel)
        lower than 5um.
    """
    threshold = (5*(3**0.5)*dx)*1.01
    return np.sum(distance[region_mask] < threshold)

def contact_area_2(region_mask, distance):
    """ Identify pixels within `region_mask` with a distance (to blood vessel)
        lower than 2um.
    """
    threshold = (2*(3**0.5)*dx)*1.01
    return np.sum(distance[region_mask] < threshold)

def contact_area_1(region_mask, distance):
    """ Identify pixels within `region_mask` with a distance (to blood vessel)
        lower than 1um.
    """
    threshold = (1*(3**0.5)*dx)*1.01
    return np.sum(distance[region_mask] < threshold)

def surface_area(region_mask):
    """ Calculate surface area of the object in `region_mask`.
    """
    integer_mask = region_mask.astype(np.int8)
    z_surf = np.sum((np.diff(integer_mask, axis=0, prepend=0, append=0) !=0))*dx*dz
    y_surf = np.sum((np.diff(integer_mask, axis=1, prepend=0, append=0) !=0))*dx*dz
    x_surf = np.sum((np.diff(integer_mask, axis=2, prepend=0, append=0) !=0))*dx*dz
    
    return z_surf + y_surf + x_surf

def overlay_probs_mask(segments, probs, figsize=(4,4), axes = None, alpha=1):
    """ Plot an overlay of Ilastik probabilities onto binary segmentations.
    """
    interpl = "none"
    if axes is None:
        fig1, axes = plt.subplots(figsize=figsize)
    axes.imshow(np.ma.masked_array(segments, segments==0).max(axis=0), interpolation=interpl, cmap="cet_glasbey_light", alpha=alpha)
    axes.imshow(1-probs.max(axis=0), interpolation=interpl, cmap="gray", alpha=0.5)
    return axes

def sharpen_watershed(ws_segments, glia_prob, n_labels):
    """ Refine watershed segmentation based on Ilastik probabilities.
    """
    ws_segments2 = np.copy(ws_segments)
    for i in np.arange(1, n_labels+1):
        try:
            isolated_prob = glia_prob[ws_segments==i]
            thresh = filters.threshold_otsu(isolated_prob)*0.8
        except IndexError:
            continue
        # Only keep pixels whose glia probability is higher than a threshold
        ws_segments2[ws_segments==i] = \
            (glia_prob[ws_segments==i]>thresh).astype(np.uint16)*i
    return ws_segments2

In [11]:
def watershed_segment(glia_prob, labels_ML, raw_img):
    """ Perform segmentation of glia from probabilities, labels, and raw images
    """
    closing_size = 3
    opening_size = 1
    dilation_size=5
    strel_dil = morphology.ball(dilation_size)
    strel_close = morphology.ball(closing_size)
    strel_open = morphology.ball(opening_size)
    
    # Spatial smoothing and downsampling
    smoothed_probs = filters.gaussian(glia_prob, sigma=7)
    scale_factor = 2
    print(smoothed_probs.shape)
    ds_sm_probs = transform.downscale_local_mean(smoothed_probs, scale_factor)
    
    # Find local maxima to serve as seeds for watershedding
    local_maxima = feature.peak_local_max(ds_sm_probs, \
        footprint = morphology.ball(10//scale_factor),\
                                      threshold_abs=0.03,\
                        min_distance=3)*scale_factor
    
    # Morphological operations to clean up spurious ROIs identified during binary segmentation
    ML_mask_aug = ndimage.binary_closing(labels_ML, strel_close)
    ML_mask_aug = ndimage.binary_dilation(ML_mask_aug, strel_close)
    removed_mask = morphology.remove_small_objects(ML_mask_aug, 100)
    segment_area = ndimage.binary_dilation(removed_mask, strel_dil)
    
    
    seeds = tuple(zip(*local_maxima))
    # seeds = tuple(zip(*ndimage.maximum_position(probs[:,:,:,label-1], \
    #             labels=rm2_labels, index=np.arange(est_object_count)+1)))

    seed_mask = np.zeros_like(glia_prob, dtype=bool)
    seed_mask[seeds] = True
    seed_labels, _ = ndimage.label(seed_mask)
    
    # Do watershedding
    ws_segments = segmentation.watershed(-glia_prob, \
                seed_labels, mask=segment_area)
    n_labels = np.max(ws_segments)
    
    # Perform watershedding
    ws_segments2 = sharpen_watershed(ws_segments, glia_prob, n_labels)
    
    # Plot the segmentation results to test 
    fig1, axes = plt.subplots(2,3,figsize=(18,12))
    axes = axes.ravel()
    axes[0].imshow(raw_img[:,0,:,:].max(axis=0))
    axes[1].imshow(glia_prob.max(axis=0))
    
    rois_ML, _ = ndimage.label(labels_ML)
    overlay_probs_mask(rois_ML, glia_prob, axes=axes[2], alpha=0.5)
    
    axes[3].imshow(smoothed_probs.max(axis=0))
    axes[3].plot(local_maxima[:,2], local_maxima[:,1], "wx")
    
    overlay_probs_mask(ws_segments, glia_prob, axes=axes[4], alpha=0.5)
    
    overlay_probs_mask(ws_segments2, glia_prob, axes=axes[5], alpha=0.5)
    
    axes[0].set_title("Raw fluorescence")
    axes[1].set_title("Ilastik probabilities")
    axes[2].set_title("Maximum likelihood")
    
    axes[3].set_title("Smoothed global maxima")
    axes[4].set_title("Watershedding")

    axes[5].set_title("Watershedding sharpened")
    return ws_segments2, fig1

Load information about experiments from file structure.

In [15]:
expt_data = parse_directories(rootdir, "^[\w]+(?=.tif)", \
                              ["(?<=%s_).+(?=.tif)","(?<=^%s_).+.h5"])
expt_data["measurements"] = None
expt_data.head(10)


,Probabilities.h5,file_path,sample_name,measurements
sample_id,,,,
/home/no221/test/Fish9,/home/no221/test/Fish9_Probabilities.h5,/home/no221/test/Fish9.tif,Fish9,None


In [12]:
# Parse directory structure to pick up tif and h5 files we don't use this anymore because we made it less complicated
expt_data = parse_directories(rootdir, "^[\w]+(?=.tif)", \
                              ["(?<=%s_).+(?=.tif)","(?<=^%s_).+.h5"])
expt_data = expt_data[pd.notna(expt_data["Probabilities.h5"])].sort_values("subfolder_0")
expt_data["measurements"] = None
del expt_data["subfolder_1"]
del expt_data["subfolder_2"]
# Get fish age from subfolder
#expt_data.rename(columns={"subfolder_0": "age"}, inplace=True)

Perform segmentation and measurements.

In [16]:
# Set matplotlib rendering method so figures don't write to the Jupyter notebook
%matplotlib agg
# for file_idx in range(1):
for file_idx in range(expt_data.shape[0]):
    sample_info = expt_data.iloc[file_idx]
    sample_name = expt_data["sample_name"].iloc[file_idx]
    if expt_data["Probabilities.h5"].isna().iloc[file_idx]:
        print(sample_name)
        continue
        
    # Make new folder for each image
    sample_path = os.path.join(rootdir, "analysis", sample_name)
    print(sample_path)

    os.makedirs(sample_path, exist_ok=True)
    
    
    expt_data["measurements"].iloc[file_idx] = os.path.join(sample_path, "measurements.csv")
    
    # Load probability files
    p = h5py.File(sample_info["Probabilities.h5"], "r")
    print(sample_info["Probabilities.h5"])
    
    probs = transform.downscale_local_mean(p['exported_data'][0], (downsample, downsample, downsample,1))
    raw_img = transform.downscale_local_mean(skio.imread(sample_info["file_path"]), (downsample, 1, downsample, downsample)).astype(np.uint8)
    
    # Separate predicted classes
    masks = simple_probabilities_to_mask_array(probs, 3)
    
    # Get distance of every pixel to blood vessels (class 1)
    distance_to_bv = ndimage.distance_transform_edt(~(masks[0]), sampling=(dz, dx, dx))
    labels = simple_mask_array_to_labels(masks, 3).astype(np.uint16)
    
    bv_labels = labels[1]
    glia_labels_ML =labels[0]
    
    # Watershed glia labels
    glia_labels_final, fig1 = watershed_segment(probs[:,:,:,1],glia_labels_ML, raw_img)
    plt.figure(fig1.number)
    plt.suptitle("%s, %s" %("", sample_name))
    
    plt.savefig(os.path.join(sample_path, "object_seg_check.tif"), dpi=250)
    
    skio.imsave(os.path.join(sample_path, "blood_vessel_labels.tif"), bv_labels)
    skio.imsave(os.path.join(sample_path, "glia_labels.tif"), glia_labels_final)
    
    # Take measurements of the segmented regions
    rps = measure.regionprops_table(glia_labels_final, intensity_image=distance_to_bv, properties=("label", "centroid", "area", \
                "intensity_mean", "intensity_min", "intensity_max"), \
                            extra_properties=[surface_area, contact_area_5, contact_area_2, contact_area_1, ellipse_3d])
    rps_df = pd.DataFrame(rps)
    rps_df[["centroid-0", "centroid-1", "centroid-2"]] *= np.array([dz, dx, dx])
    rps_df[["area"]] *= dz*dx**2
    rps_df[["contact_area_5"]] *= dz*dx
    rps_df[["contact_area_2"]] *= dz*dx
    rps_df[["contact_area_1"]] *= dz*dx
    
    rps_df.rename(columns={"ellipse_3d-0": "ellipse_ax1", "ellipse_3d-1": "ellipse_ax2",\
                           "ellipse_3d-2": "ellipse_ax3", "ellipse_3d-3": "ellipse_surface_area",\
                          "ellipse_3d-4": "ellipse_volume", "area":"volume",\
"intensity_mean": "dist_bv_mean", "intensity_min": "dist_bv_min", "intensity_max": "dist_bv_max",\
                  "centroid-0": "centroid_z", "centroid-1": "centroid_y", "centroid-2": "centroid_x"},\
                  inplace=True)
    
    rps_df["eccentricity"] = np.sqrt(1 - (rps_df["ellipse_ax1"]/rps_df["ellipse_ax2"])**2)
    rps_df["bushiness_1"] = rps_df["surface_area"]/rps_df["ellipse_surface_area"]
    rps_df["bushiness_2"] = rps_df["surface_area"]/rps_df["volume"]/(rps_df["ellipse_surface_area"]/rps_df["ellipse_volume"])
    rps_df["sample_name"] = sample_name
#     rps_df["age"] = sample_info["age"]
    
    rps_df.to_csv(os.path.join(sample_path, "measurements.csv"), index=False)
    

/home/no221/test/analysis/Fish9
/home/no221/test/Fish9_Probabilities.h5
(173, 512, 512)


/tmp/ipykernel_21013/123272062.py:44: UserWarning: /home/no221/test/analysis/Fish9/blood_vessel_labels.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "blood_vessel_labels.tif"), bv_labels)
/tmp/ipykernel_21013/123272062.py:45: UserWarning: /home/no221/test/analysis/Fish9/glia_labels.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "glia_labels.tif"), glia_labels_final)
/home/no221/.conda/envs/glia_v3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Combine measurements for all cells in each sample

In [13]:
all_dfs = []
# for file_idx in [0]:
for file_idx in range(expt_data.shape[0]):
    sample_info = expt_data.iloc[file_idx]
    sample_name = expt_data["sample_name"].iloc[file_idx]
    sample_path = os.path.join(rootdir, "analysis", sample_name)
    rps_df = pd.read_csv(os.path.join(sample_path, "measurements.csv"))
    all_dfs.append(rps_df)
all_dfs_pd = pd.concat(all_dfs, axis=0)
all_dfs_pd.to_csv(os.path.join(rootdir, "analysis", "all_glia_features.csv"), index=False)
expt_data.to_csv(os.path.join(rootdir, "analysis", "expt_metadata.csv"), index=False)

Put number labels on segmentations

In [14]:
font = cv2.FONT_HERSHEY_SIMPLEX

# fontScale
fontScale = 0.4
  
# Line thickness of 2 px
thickness = 1

In [15]:
# for file_idx in range(1):
for file_idx in range(expt_data.shape[0]):
    # Load images
    sample_info = expt_data.iloc[file_idx]
    sample_name = expt_data["sample_name"].iloc[file_idx]
    sample_path = os.path.join(rootdir, "analysis", sample_name)
    glia_labels_final = skio.imread(os.path.join(sample_path, "glia_labels.tif")).astype(np.uint16)
    
    # Load measurements
    rps_df = pd.read_csv(os.path.join(sample_path, "measurements.csv"))
    
    # Find locations of segmented cells and convert centroids back to pixels
    xy_centroid_labels = rps_df[["label"]].copy()
    xy_centroid_labels["centroid_y_px"] = (rps_df["centroid_y"]//dx).astype(int)
    xy_centroid_labels["centroid_x_px"] = (rps_df["centroid_x"]//dx).astype(int)
    xy_centroid_labels = xy_centroid_labels.set_index("label")
    
    # convert label image to RGB
    glia_labels_rgb = exposure.rescale_intensity(color.label2rgb(glia_labels_final, alpha=1), out_range=np.uint8)
    blank = np.zeros_like(glia_labels_rgb, dtype=np.uint8)
    
    # Draw text for each cell
    for z in range(glia_labels_rgb.shape[0]):
        unique_labels = np.unique(glia_labels_final[z])
        if len(unique_labels) > 1:
            for l in unique_labels[1:]:
                glia_labels_rgb[z] = cv2.putText(glia_labels_rgb[z], str(l),\
                                                 tuple(xy_centroid_labels.loc[l][["centroid_x_px", "centroid_y_px"]]),\
                                                font, fontScale, (255, 255, 255), thickness, cv2.LINE_AA)
                blank[z] = cv2.putText(blank[z], str(l),\
                                 tuple(xy_centroid_labels.loc[l][["centroid_x_px", "centroid_y_px"]]),\
                                font, fontScale, (255, 255, 255), thickness, cv2.LINE_AA)

    
    glia_labels_numbered_channel = np.moveaxis(np.array([glia_labels_final, blank.max(axis=-1).astype(np.uint16)]), 1, 0)
    # Save RGB version and version where labels are a separate image channel
    skio.imsave(os.path.join(sample_path, "glia_labels_numbered_rgb_small.tif"), glia_labels_rgb)
    skio.imsave(os.path.join(sample_path, "glia_labels_numbered_channel_small.tif"), glia_labels_numbered_channel)

/tmp/ipykernel_25198/677993633.py:37: UserWarning: /home/no221/3dpf/analysis/Fish1/glia_labels_numbered_rgb_small.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "glia_labels_numbered_rgb_small.tif"), glia_labels_rgb)
/tmp/ipykernel_25198/677993633.py:38: UserWarning: /home/no221/3dpf/analysis/Fish1/glia_labels_numbered_channel_small.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "glia_labels_numbered_channel_small.tif"), glia_labels_numbered_channel)
/tmp/ipykernel_25198/677993633.py:37: UserWarning: /home/no221/3dpf/analysis/Fish10/glia_labels_numbered_rgb_small.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "glia_labels_numbered_rgb_small.tif"), glia_labels_rgb)
/tmp/ipykernel_25198/677993633.py:38: UserWarning: /home/no221/3dpf/analysis/Fish10/glia_labels_numbered_channel_small.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "glia_labels_numbered_channel_small.tif"), glia_labels_numbered_channel)
/tmp

Generate contact masks

In [16]:
%matplotlib agg

for file_idx in range(expt_data.shape[0]):
    sample_info = expt_data.iloc[file_idx]
    sample_name = expt_data["sample_name"].iloc[file_idx]
    sample_path = os.path.join(rootdir, "analysis", sample_name)
    print(sample_path)
    bv_labels = skio.imread(os.path.join(sample_path, "blood_vessel_labels.tif"))
    glia_labels_final = skio.imread(os.path.join(sample_path, "glia_labels.tif"))
    bv_mask = bv_labels > 0
    
    distance_to_bv = ndimage.distance_transform_edt(~bv_mask, sampling=(dz, dx, dx))
    # Generate mask of pixels closer to blood vessels than a defined threshold. 
    # Also account for pixels that are diagonally touching.
    threshold = 1
    contact_mask = (distance_to_bv < (threshold*(3**0.5)*dx)*1.01) * (glia_labels_final > 0)
    skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)

/home/no221/3dpf/analysis/Fish1


/tmp/ipykernel_25198/2305950624.py:17: UserWarning: /home/no221/3dpf/analysis/Fish1/contact_mask.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)
/tmp/ipykernel_25198/2305950624.py:17: UserWarning: /home/no221/3dpf/analysis/Fish1/contact_mask.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)


/home/no221/3dpf/analysis/Fish10


/tmp/ipykernel_25198/2305950624.py:17: UserWarning: /home/no221/3dpf/analysis/Fish10/contact_mask.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)
/tmp/ipykernel_25198/2305950624.py:17: UserWarning: /home/no221/3dpf/analysis/Fish10/contact_mask.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)


/home/no221/3dpf/analysis/Fish11


/tmp/ipykernel_25198/2305950624.py:17: UserWarning: /home/no221/3dpf/analysis/Fish11/contact_mask.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)
/tmp/ipykernel_25198/2305950624.py:17: UserWarning: /home/no221/3dpf/analysis/Fish11/contact_mask.tif is a low contrast image
  skio.imsave(os.path.join(sample_path, "contact_mask.tif"), contact_mask)
